In [1]:
import pyawd
import numpy as np
import sktime
import matplotlib.pyplot as plt

In [2]:
X_train = np.load("train2D_interrogators_data.npy")[:, 0]
Y_train = np.load("train2D_epicenters.npy")

X_test = np.load("test2D_interrogators_data.npy")[:, 0]
Y_test = np.load("test2D_epicenters.npy")

X_train = X_train.reshape(X_train.shape[0], -1, X_train.shape[-1])
X_test = X_test.reshape(X_test.shape[0], -1, X_test.shape[-1])

print("Training input shape:", X_train.shape)
print("Training output shape:", Y_train.shape)
print("Testing input shape:", X_test.shape)
print("Testing output shape:", Y_test.shape)

Training input shape: (1024, 4, 1000)
Training output shape: (1024, 2)
Testing input shape: (256, 4, 1000)
Testing output shape: (256, 2)


In [3]:
def mse(y, y_hat):
    return np.mean((y-y_hat)**2, axis=0), np.mean((y-y_hat)**2)

In [73]:
from sktime.regression.dummy import DummyRegressor
m0 = [DummyRegressor().fit(X_train, Y_train[:, 0]), DummyRegressor().fit(X_train, Y_train[:, 1])]

Y_hat = np.array(list(zip(m0[0].predict(X_test), m0[1].predict(X_test))))

error_by_coordinate, error = mse(Y_test, Y_hat)
print(f"MSE x0: %.5f,"%error_by_coordinate[0], f"MSE y0: %.5f,"%error_by_coordinate[1], f"MSE: %.5f"%error)

MSE x0: 0.33533, MSE y0: 0.33102, MSE: 0.33317


In [77]:
from sktime.regression.deep_learning import FCNRegressor
m1 = [CNNRegressor(n_epochs=10, batch_size=8, activation="tanh"), CNNRegressor(n_epochs=10, batch_size=8, activation="tanh")]
m1[0].verbose = 0
m1[1].verbose = 0
m1[0].fit(X_train, Y_train[:, 0])
m1[1].fit(X_train, Y_train[:, 1])

Y_hat = np.array(list(zip(m1[0].predict(X_test), m1[1].predict(X_test))))

error_by_coordinate, error = mse(Y_test, Y_hat)
print(f"MSE x0: %.5f,"%error_by_coordinate[0], f"MSE y0: %.5f,"%error_by_coordinate[1], f"MSE: %.5f"%error)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
MSE x0: 0.33378, MSE y0: 0.33052, MSE: 0.33215


In [4]:
import os

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [6]:
from sktime.regression.deep_learning import TapNetRegressor
m2 = [TapNetRegressor(n_epochs=10, batch_size=8), TapNetRegressor(n_epochs=10, batch_size=8)]
m2[0].verbose = 1
m2[1].verbose = 1
m2[0].fit(X_train, Y_train[:, 0])
m2[1].fit(X_train, Y_train[:, 1])

Y_hat = np.array(list(zip(m2[0].predict(X_test), m2[1].predict(X_test))))

error_by_coordinate, error = mse(Y_test, Y_hat)
print(f"MSE x0: %.5f,"%error_by_coordinate[0], f"MSE y0: %.5f,"%error_by_coordinate[1], f"MSE: %.5f"%error)

NotImplementedError: The operator 'aten::linalg_qr.out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.